In [1]:
!pip install -U gdown

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1-r4LHN_PaRVBROogfXF3VXWTWIyUoSLs/view?usp=sharing
!unzip -q FishImgDataset-20250613T014653Z-1-001.zip -d /content/

Downloading...
From (original): https://drive.google.com/uc?id=1-r4LHN_PaRVBROogfXF3VXWTWIyUoSLs
From (redirected): https://drive.google.com/uc?id=1-r4LHN_PaRVBROogfXF3VXWTWIyUoSLs&confirm=t&uuid=3eb3e43a-45cc-48b1-bd4b-dba40aed527b
To: /content/FishImgDataset-20250613T014653Z-1-001.zip
100% 1.72G/1.72G [00:21<00:00, 79.4MB/s]


# IMPORT

In [1]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
from pathlib import Path


# LOAD DATA

In [3]:
# from pathlib import Path

# # Gunakan Pathlib (lebih aman & fleksibel)
# base_dir = Path('/content/FishImgDataset')
# train_dir = base_dir / 'train'
# val_dir = base_dir / 'val'
# test_dir = base_dir / 'test'

# print("Train directory:", train_dir)
# print("Validation directory:", val_dir)
# print("Test directory:", test_dir)

Train directory: /content/FishImgDataset/train
Validation directory: /content/FishImgDataset/val
Test directory: /content/FishImgDataset/test


In [2]:
# --- 1. PERSIAPAN DATA ---
# Perintah untuk mengunduh dataset dari Google Drive (membutuhkan gdown)
# Pastikan Anda sudah menginstal gdown dengan `pip install gdown`
print("Mempersiapkan dataset...")
os.system("gdown --id 1-r4LHN_PaRVBROogfXF3VXWTWIyUoSLs -O FishImgDataset.zip")

# Ekstrak file zip ke direktori tujuan
with zipfile.ZipFile("FishImgDataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")
print("Dataset berhasil diekstrak.")

# Tentukan path direktori
base_dir = Path('/content/dataset/FishImgDataset')
train_dir = base_dir / 'train'
val_dir = base_dir / 'val'
test_dir = base_dir / 'test'

Mempersiapkan dataset...
Dataset berhasil diekstrak.


# PREPROCESSING & AUGMENTASI DATA

In [3]:
# --- 2. PREPROCESSING & AUGMENTASI DATA ---
print("Membuat data generator...")
# Parameter
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

# Augmentasi untuk data training untuk meningkatkan generalisasi model
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Untuk data validasi dan tes, hanya lakukan rescale
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Dapatkan jumlah kelas dari generator
NUM_CLASSES = len(train_generator.class_indices)

Membuat data generator...
Found 8801 images belonging to 31 classes.
Found 2751 images belonging to 31 classes.


# CREATING CNN MODEL ARCHITECTURE

In [4]:
# --- 3. MEMBANGUN ARSITEKTUR MODEL CNN ---
print("Membangun model CNN...")
model = Sequential([
    # Layer Konvolusi 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
           kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Layer Konvolusi 2
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Layer Konvolusi 3
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Flattening
    Flatten(),

    # Dense Layer
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    # Output Layer
    Dense(NUM_CLASSES, activation='softmax')
])

Membangun model CNN...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# COMPILATION & TRAINING MODEL

In [5]:
# --- 4. KOMPILASI DAN PELATIHAN MODEL ---
print("Melakukan kompilasi model...")
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Callbacks untuk mengoptimalkan pelatihan
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

print("Memulai pelatihan model...")
history = model.fit(
    train_generator,
    epochs=100, # Atur epoch tinggi, EarlyStopping akan berhenti jika sudah optimal
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)
print("Pelatihan model selesai.")

Melakukan kompilasi model...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 31)             │        15,903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,052,511 (72.68 MB)

 Trainable params: 19,051,039 (72.67 MB)

 Non-trainable params: 1,472 (5.75 KB)

Memulai pelatihan model...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 113s 373ms/step - accuracy: 0.1585 - loss: 5.4508 - val_accuracy: 0.1389 - val_loss: 4.7658 - learning_rate: 0.0010
Epoch 2/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 95s 344ms/step - accuracy: 0.2537 - loss: 4.3201 - val_accuracy: 0.2719 - val_loss: 3.8884 - learning_rate: 0.0010
Epoch 3/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 101s 365ms/step - accuracy: 0.2837 - loss: 3.8923 - val_accuracy: 0.2625 - val_loss: 4.0296 - learning_rate: 0.0010
Epoch 4/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 95s 345ms/step - accuracy: 0.3264 - loss: 3.7064 - val_accuracy: 0.2795 - val_loss: 3.9325 - learning_rate: 0.0010
Epoch 5/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 93s 339ms/step - accuracy: 0.3615 - loss: 3.5452 - val_accuracy: 0.3577 - val_loss: 3.5108 - learning_rate: 0.0010
Epoch 6/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 93s 339ms/step - accuracy: 0.3694 - loss: 3.5332 - val_accuracy: 0.3889 - val_loss: 3.5640 - learning_rate: 0.0010
Epoch 7/100
276/276 ━━━━━━━━━━━━━━━━━━━━ 94s 340ms/step - accu

# EVALUASI MODEL

In [6]:
# --- 5. EVALUASI MODEL ---
print("Melakukan evaluasi model pada data tes...")
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1, # Batch size 1 untuk evaluasi agar mudah dianalisis
    shuffle=False,
    class_mode='categorical'
)

# Lakukan prediksi
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Tampilkan Laporan Klasifikasi (Akurasi, Presisi, Recall, F1-Score)
print("\n--- Laporan Klasifikasi ---\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

Melakukan evaluasi model pada data tes...
Found 1760 images belonging to 31 classes.
1760/1760 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step

--- Laporan Klasifikasi ---

                       precision    recall  f1-score   support

               Bangus       0.64      0.62      0.63        34
        Big Head Carp       0.84      0.63      0.72        43
   Black Spotted Barb       0.87      0.82      0.85        40
              Catfish       0.74      0.73      0.73        62
       Climbing Perch       0.83      0.63      0.72        30
 Fourfinger Threadfin       0.84      0.84      0.84        38
       Freshwater Eel       0.70      0.80      0.75        55
       Glass Perchlet       0.96      0.87      0.91        77
                 Goby       0.82      0.83      0.82       124
            Gold Fish       1.00      1.00      1.00        41
              Gourami       0.97      0.95      0.96        63
           Grass Carp       0.87      0.97      0.92       238
 Green Spotted Puffer

# VISUALISASI HASIL

In [9]:
# --- 6. VISUALISASI HASIL ---
print("Menyimpan hasil visualisasi...")
# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('Kelas Aktual')
plt.xlabel('Kelas Prediksi')
plt.savefig('confusion_matrix.png')
plt.close()
print("Confusion Matrix disimpan sebagai 'confusion_matrix.png'")

# Grafik Akurasi dan Loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Akurasi Training')
plt.plot(epochs_range, val_acc, label='Akurasi Validasi')
plt.legend(loc='lower right')
plt.title('Akurasi Training dan Validasi')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Loss Training')
plt.plot(epochs_range, val_loss, label='Loss Validasi')
plt.legend(loc='upper right')
plt.title('Loss Training dan Validasi')
plt.savefig('training_history.png')
plt.close()
print("Grafik riwayat pelatihan disimpan sebagai 'training_history.png'")

# Kurva AUC-ROC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(NUM_CLASSES):
    # Handle cases where a class might not be present in y_true or y_pred
    if np.sum(y_true == i) == 0 or np.sum(y_pred == i) == 0:
        fpr[i] = [0.0, 1.0]
        tpr[i] = [0.0, 1.0]
        roc_auc[i] = np.nan # Use NaN or 0 if AUC can't be calculated
    else:
        fpr[i], tpr[i], _ = roc_curve(y_true == i, y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])


plt.figure(figsize=(10, 8))
colors = plt.cm.get_cmap('tab10', NUM_CLASSES).colors # Access the colors attribute
for i, color in zip(range(NUM_CLASSES), colors):
    # Only plot if AUC was calculable
    if not np.isnan(roc_auc[i]):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'Kurva ROC kelas {class_labels[i]} (area = {roc_auc[i]:.2f})')
    else:
         plt.plot([], [], ' ', label=f'Kurva ROC kelas {class_labels[i]} (Tidak Dapat Dihitung)') # Add a placeholder in legend

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.savefig('roc_curve.png')
plt.close()
print("Kurva AUC-ROC disimpan sebagai 'roc_curve.png'")

Menyimpan hasil visualisasi...
Confusion Matrix disimpan sebagai 'confusion_matrix.png'
Grafik riwayat pelatihan disimpan sebagai 'training_history.png'


/tmp/ipython-input-9-3469088151.py:54: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', NUM_CLASSES).colors # Access the colors attribute


Kurva AUC-ROC disimpan sebagai 'roc_curve.png'


CATATAN HASIL VISUALISASI DI DOWNLOAD